In [1]:
import ctypes
import numpy as np

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=128)

In [4]:
inp=np.random.randn(128,64,64,32).astype(np.float32)

In [5]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col)).astype(np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

In [6]:
img=padded[0]

In [7]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [22]:
%%time
output=np.empty((batches,out_row*out_col,num_ker),dtype=np.float32)
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
# output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 3.01 s, sys: 220 ms, total: 3.23 s
Wall time: 844 ms


In [9]:
%%time
coled=np.empty((batches,*ind.shape),dtype=np.float32)#,order='F')
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    coled[i]=img.take(ind)
# output+=b0
# ans2=output.reshape(batches,out_row,out_col,num_ker)
# output=(np.dot(np.take(padded, bind), self.kern)+self.biases)
# print(coled.nbytes/1024/1024)

CPU times: user 255 ms, sys: 39.9 ms, total: 295 ms
Wall time: 294 ms


In [10]:
%%time
a=coled.reshape(-1,d*ksz*ksz)
b=kern.reshape(d*ksz*ksz,-1)
(a.nbytes+b.nbytes)/1024/1024

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 15.7 µs


In [25]:
c=np.empty((a.shape[0],b.shape[1]),dtype=np.float32,order='F')
c.nbytes/1024/1024

256.0

In [26]:
%%time
a=np.asfortranarray(a)
b=np.asfortranarray(b)
c=np.asfortranarray(c)

CPU times: user 13 µs, sys: 1e+03 ns, total: 14 µs
Wall time: 17.2 µs


In [27]:
np.isfortran(c)

True

In [28]:
c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
sgemm=ctypes.CDLL('libsgemm.so')

In [16]:
al,bet=1,1

In [17]:
%%time
sgemm.gemm(ctypes.c_void_p(a.ctypes.data),ctypes.c_void_p(b.ctypes.data),ctypes.c_void_p(c.ctypes.data),a.shape[0],b.shape[0],c.shape[1],ctypes.c_float(al),ctypes.c_float(bet),ctypes.c_void_p(b0[0].ctypes.data))

CPU times: user 655 ms, sys: 345 ms, total: 1e+03 ms
Wall time: 1 s


0

In [18]:
a.shape[0],b.shape[0],c.shape[1]

(524288, 288, 128)

In [19]:
# al*np.dot(a,b)+bet*c

In [20]:
c

array([[ 0.04752279,  0.01789882,  0.22528622, ...,  0.08359388,
        -0.11860363,  0.03961049],
       [ 0.15305048,  0.06723985, -0.2359069 , ...,  0.33198187,
        -0.12861888, -0.10043315],
       [ 0.10796219,  0.1838205 , -0.20515937, ...,  0.04283091,
         0.13516037, -0.20007604],
       ...,
       [ 0.09197106,  0.16292071, -0.19378963, ...,  0.06512031,
        -0.05821431, -0.14989364],
       [ 0.12678374, -0.16552833, -0.01943905, ..., -0.22675996,
        -0.13070531, -0.11424434],
       [-0.08691256, -0.18982828, -0.14254744, ..., -0.0224365 ,
        -0.09677073,  0.05016772]], dtype=float32)

In [23]:
ans2.reshape(-1,128)

array([[ 0.04752279,  0.01789882,  0.22528622, ...,  0.08359388,
        -0.11860363,  0.03961049],
       [ 0.15305048,  0.06723985, -0.2359069 , ...,  0.33198187,
        -0.12861888, -0.10043315],
       [ 0.10796219,  0.1838205 , -0.20515937, ...,  0.04283091,
         0.13516037, -0.20007604],
       ...,
       [ 0.09197106,  0.16292071, -0.19378963, ...,  0.06512031,
        -0.05821431, -0.14989364],
       [ 0.12678374, -0.16552833, -0.01943905, ..., -0.22675996,
        -0.13070531, -0.11424434],
       [-0.08691256, -0.18982828, -0.14254744, ..., -0.0224365 ,
        -0.09677073,  0.05016772]], dtype=float32)

In [24]:
(c==ans2.reshape(-1,128)).all()

True

In [28]:
a=(np.arange(30)+11).reshape(6,5,order='F').astype(np.float32)  #mxk

In [29]:
b=(np.arange(20)+11).reshape(5,4,order='F').astype(np.float32)  #kxn

In [30]:
c=(np.arange(24)*0).reshape(6,4,order='F').astype(np.float32)  #mxn